In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

In [2]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from langsmith import traceable, trace

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Answer all the questions asked by the user diligently.
If you don't know the answer, just say that you don't know.
"""

openai_client = OpenAI()
nest_asyncio.apply()

def generate_response(question: str):
    with trace(
        name="Generate Response", 
        run_type="chain",
        inputs={"question": question},
        metadata={"foo": "bar"},
    ) as ls_trace:
        messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Question: {question}"
        }
        ]
        response = call_openai(messages)
        ls_trace.end(outputs={"output": response})
    
    return response

@traceable()
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    response = generate_response(question)
    return response.choices[0].message.content

@traceable
def chat_bot():
    print("This is a simple chatbot. Type 'byee' to quit.\n")
    
    while True:
        ques = input("Ask Question: ")
        print(f"Question: {ques}")
        
        if ques.lower() == 'byee':
            print("bei")
            break
        
        response = langsmith_rag(ques)
        print(f"AI: {response}\n")


chat_bot()

This is a simple chatbot. Type 'byee' to quit.

Question: in which galaxy do we live
AI: We live in the Milky Way galaxy.

Question: byee
bei


In [ ]:
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from langsmith import traceable, trace
from langsmith.wrappers import wrap_openai

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Answer all the questions asked by the user diligently.
If you don't know the answer, just say that you don't know.
"""

openai_client = wrap_openai(OpenAI())
nest_asyncio.apply()

def generate_response(question: str):
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Question: {question}"
        }
    ]
    
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )
    
    return response

@traceable()
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def chat_bot():
    print("This is a simple chatbot. Type 'byee' to quit.\n")
    
    while True:
        ques = input("Enter the question: ")
        print(f"Question: {ques}")
        
        if ques.lower() == 'byee':
            print("au revoir")
            break
        
        response = langsmith_rag(ques)
        print(f"AI: {response}\n")


chat_bot()

This is a simple chatbot. Type 'exit' to quit.

Question: whos chacha chaudhar
AI: Chacha Chaudhary is a popular Indian comic book character created by cartoonist Pran Kumar Sharma. He is known for his wit, intelligence, and resourcefulness. Chacha Chaudhary is depicted as an elderly man with a distinct white beard and a red turban, who, with the help of his giant friend Sabu, solves various problems and fights against injustice. The stories often incorporate humor and moral lessons, making Chacha Chaudhary a beloved figure in Indian pop culture.

Question: byee
AI: Goodbye! If you have more questions in the future, feel free to ask. Take care!

Question: exit
Exiting the chatbot. Goodbye!
